## 初期設定

In [1]:
from neo4j_test import HelloWorldExample, with_neo4j_connection

# デコレータを使用した関数の定義
@with_neo4j_connection
def execute_query(greeter, query):
    greeter.query_execute(query)

@with_neo4j_connection
def clear_database(greeter):
    greeter.clear_db()

@with_neo4j_connection
def execute_query_and_return_string(greeter, query):
    return greeter.execute_query_and_return_string(query)

clear_database()

要件：
- シェイクスピアという作家がいる
- ジュリウス・カエサルという劇がある
- シェイクスピアがジュリウス・カエサルを書いた
- テンペストという劇がある
- シェイクスピアがテンペストを書いた
- RSCという会社がある
- ジュリウス・カエサルのプロダクションがある
- RSCがジュリウス・カエサルのプロダクションを製作
- ジュリウス・カエサルのプロダクションがジュリウス・カエサルの劇
- 2012年7月29日にジュリウス・カエサルのパフォーマンスがあった
- ジュリウス・カエサルのパフォーマンスがジュリウス・カエサルのプロダクションに関連
- テンペストのプロダクションがある
- RSCがテンペストのプロダクションを製作
- テンペストのプロダクションがテンペストの劇
- 2006年11月21日にテンペストのパフォーマンスがあった
- テンペストのパフォーマンスがテンペストのプロダクションに関連
- 2012年7月30日にジュリウス・カエサルのパフォーマンスがあった
- ジュリウス・カエサルのパフォーマンスがジュリウス・カエサルのプロダクションに関連
- Billyというユーザーがいる
- レビューがある
- Billyがレビューを書いた
- レビューがジュリウス・カエサルのパフォーマンスに関連
- Theatre Royalという場所がある
- ジュリウス・カエサルのパフォーマンスがTheatre Royalで行われた
- テンペストのパフォーマンスがTheatre Royalで行われた
- ジュリウス・カエサルのパフォーマンスがTheatre Royalで行われた
- Grey Streetという街がある
- シェイクスピアがStratfordで生まれた

### SQLで書いたらこうなります

DDL
```
CREATE TABLE Authors (
    id INT PRIMARY KEY AUTO_INCREMENT,
    firstname VARCHAR(255),
    lastname VARCHAR(255)
);

CREATE TABLE Plays (
    id INT PRIMARY KEY AUTO_INCREMENT,
    title VARCHAR(255)
);

CREATE TABLE Companies (
    id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(255)
);

CREATE TABLE Productions (
    id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(255),
    play_id INT,
    FOREIGN KEY (play_id) REFERENCES Plays(id)
);

CREATE TABLE Performances (
    id INT PRIMARY KEY AUTO_INCREMENT,
    date DATE,
    production_id INT,
    FOREIGN KEY (production_id) REFERENCES Productions(id)
);

CREATE TABLE Users (
    id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(255)
);

CREATE TABLE Reviews (
    id INT PRIMARY KEY AUTO_INCREMENT,
    rating INT,
    review TEXT,
    user_id INT,
    performance_id INT,
    FOREIGN KEY (user_id) REFERENCES Users(id),
    FOREIGN KEY (performance_id) REFERENCES Performances(id)
);

CREATE TABLE Venues (
    id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(255)
);

CREATE TABLE Streets (
    id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(255)
);

CREATE TABLE Birthplaces (
    id INT PRIMARY KEY AUTO_INCREMENT,
    author_id INT,
    street_id INT,
    FOREIGN KEY (author_id) REFERENCES Authors(id),
    FOREIGN KEY (street_id) REFERENCES Streets(id)
);

```

```
DML
-- Authorsテーブルにシェイクスピアを挿入
INSERT INTO Authors (firstname, lastname) VALUES ('William', 'Shakespeare');

-- Playsテーブルにジュリウス・カエサルとテンペストを挿入
INSERT INTO Plays (title) VALUES ('Julius Caesar'), ('The Tempest');

-- CompaniesテーブルにRSCを挿入
INSERT INTO Companies (name) VALUES ('RSC');

-- Productionsテーブルにジュリウス・カエサルとテンペストのプロダクションを挿入
INSERT INTO Productions (name, play_id) VALUES ('Julius Caesar', 1), ('The Tempest', 2);

-- Performancesテーブルにパフォーマンスを挿入
INSERT INTO Performances (date, production_id) VALUES ('2012-07-29', 1), ('2006-11-21', 2), ('2012-07-30', 1);

-- UsersテーブルにBillyを挿入
INSERT INTO Users (name) VALUES ('Billy');

-- Reviewsテーブルにレビューを挿入
INSERT INTO Reviews (rating, review, user_id, performance_id) VALUES (5, 'This was awesome!', 1, 1);

-- VenuesテーブルにTheatre Royalを挿入
INSERT INTO Venues (name) VALUES ('Theatre Royal');

-- StreetsテーブルにGrey Streetを挿入
INSERT INTO Streets (name) VALUES ('Grey Street');

-- Birthplacesテーブルにシェイクスピアの誕生地を挿入
INSERT INTO Birthplaces (author_id, street_id) VALUES (1, 1);
```


### DDL　with Cipher

In [2]:
execute_query(
"""
  CREATE (shakespeare:Author {firstname:'William', lastname:'Shakespeare'}), 
       (juliusCaesar:Play {title:'Julius Caesar'}), 
       (shakespeare)-[:WROTE_PLAY {year:1599}]->(juliusCaesar), 
       (theTempest:Play {title:'The Tempest'}), 
       (shakespeare)-[:WROTE_PLAY {year:1610}]->(theTempest), 
       (rsc:Company {name:'RSC'}),
       (production1:Production {name:'Julius Caesar'}),
       (rsc)-[:PRODUCED]->(production1),
       (production1)-[:PRODUCTION_OF]->(juliusCaesar),
       (performance1:Performance {date:20120729}),
       (performance1)-[:PERFORMANCE_OF]->(production1),
       (production2:Production {name:'The Tempest'}),
       (rsc)-[:PRODUCED]->(production2),
       (production2)-[:PRODUCTION_OF]->(theTempest),
       (performance2:Performance {date:20061121}),
       (performance2)-[:PERFORMANCE_OF]->(production2),
       (performance3:Performance {date:20120730}),
       (performance3)-[:PERFORMANCE_OF]->(production1),
       (billy:User {name:'Billy'}),
       (review:Review {rating:5, review:'This was awesome!'}),
       (billy)-[:WROTE_REVIEW]->(review),
       (review)-[:RATED]->(performance1),
       (theatreRoyal:Venue {name:'Theatre Royal'}),
       (performance1)-[:VENUE]->(theatreRoyal),
       (performance2)-[:VENUE]->(theatreRoyal),
       (performance3)-[:VENUE]->(theatreRoyal),
       (greyStreet:Street {name:'Grey Street'}),
       (shakespeare)-[:BORN_IN]->(greyStreet);
""")


### 返却結果を辞書型で返却します

In [8]:

# このクエリは、William Shakespeareが生まれた場所を返します。
result = execute_query_and_return_string(
"""
MATCH (a:Author {firstname:'William', lastname:'Shakespeare'})-[:BORN_IN]->(s:Street)
RETURN s.name
"""
)
print(result)

# このクエリは、Billyが書いたレビューを返します。
result = execute_query_and_return_string(
"""
MATCH (u:User {name:'Billy'})-[:WROTE_REVIEW]->(r:Review)
RETURN r.review
"""
)
print(result)

# このクエリは、Billyが書いたレビューと評価を返します。
result = execute_query_and_return_string(
"""
MATCH (u:User {name:'Billy'})-[:WROTE_REVIEW]->(r:Review)-[:RATED]->(p:Performance)
RETURN r.review, r.rating
"""
)
print(result)


# このクエリは、Theatre Royalで行われたパフォーマンスの元となった劇を返します。
result = execute_query_and_return_string(
"""
MATCH (v:Venue {name:'Theatre Royal'})<-[:VENUE]-(p:Performance)-[:PERFORMANCE_OF]->(pr:Production)
RETURN pr
"""
)
print(result)

{'s.name': 'Grey Street'}
{'r.review': 'This was awesome!'}
{'r.review': 'This was awesome!', 'r.rating': 5}
{'pr': {'name': 'Julius Caesar'}}


### お片付け

In [2]:
clear_database()